In [2]:
import torch
import torchvision # provide access to datasets, models, transforms, utils, etc
import torchvision.transforms as transforms
import torch.optim as optim
import random
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F

In [3]:
def Rand(start, end, num):
    res = []
    for j in range(num):
        res.append(random.random(start, end)) 
    return res

In [4]:
# let's extend the Dataset class of Pytorch to build our own sample dataset
class rand_num(torch.utils.data.Dataset):
  def __init__(self):
    self.data = torch.randint(0,9,(60000,),requires_grad=True,dtype=torch.float32)

  def __getitem__(self, index):
    r = self.data[index]
    return r
  
  def __len__(self):
    return len(self.data)

In [5]:
df1 = rand_num()

In [6]:
train_set = torchvision.datasets.MNIST(
    root='./files'
    ,train=True
    ,download=True
    ,transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

In [7]:
class ImageDataset(Dataset):
    """Tabular and Image dataset."""

    def __init__(self, images, num, labels=None, transforms=None):
        self.X = images
        self.X2 = num

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        data = self.X[i][0]
        data2 = self.X2[i]
        Y = self.X[i][1]
        Y2 = self.X2[i] + self.X[i][1]

        return data,data2,Y, Y2

In [8]:
data_combined = ImageDataset(train_set,df1)

In [9]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        self.fc1 = nn.Linear(in_features=12 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out1 = nn.Linear(in_features=60, out_features=10)

        self.fc3 = nn.Linear(2,2)
        self.fc4 = nn.Linear(5, 2)
        self.out2 = nn.Linear(2, 1)

    
    def forward(self, t,tab):
        # (1) input layer
        t = t

        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        t = self.fc1(t)
        t = F.relu(t)

        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)

        # (6) output layer
        t = self.out1(t)
        t1 = F.softmax(t, dim=1)
        
        inp1 = torch.argmax(t1,dim=1)
        x = torch.stack((inp1,tab),dim=1).type(torch.float32) #nx2
        # x = self.fc3(x)
        # x = F.relu(x)
        # x = self.fc4(x)
        # x = F.relu(x)
        t2 = self.out2(x)#2x1
        return t1, t2

In [10]:
model = Network()

In [11]:
model

Network(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 12, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=192, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=60, bias=True)
  (out1): Linear(in_features=60, out_features=10, bias=True)
  (fc3): Linear(in_features=2, out_features=2, bias=True)
  (fc4): Linear(in_features=5, out_features=2, bias=True)
  (out2): Linear(in_features=2, out_features=1, bias=True)
)

In [12]:
def get_num_correct(preds, labels):
  return preds.argmax(dim=1).eq(labels).sum().item()

In [13]:
torch.set_grad_enabled(True)
network = Network()

if torch.cuda.is_available():
    network.cuda()

train_loader = torch.utils.data.DataLoader(data_combined, batch_size=32)
optimizer = optim.Adam(network.parameters(), lr=0.001)

for epoch in range(100):

    total_loss_1 = 0
    total_loss_2 = 0
    total_correct = 0

    for batch in train_loader: # Get Batch
        images, data, y, y_sum = batch 

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        images, data, y, y_sum = images.to(device), data.to(device),y.to(device), y_sum.to(device)

        preds = network(images,data) # Pass Batch
        b = nn.MSELoss()
        a = nn.CrossEntropyLoss()

        loss_a = a(preds[0],y)
        loss_b = b(preds[1], y_sum)
        loss = loss_a + loss_b

        optimizer.zero_grad()
        loss_a.backward() # Calculate Gradients
        loss_b.backward()
        optimizer.step() # Update Weights

        total_loss_1 += loss_a.item()
        total_loss_2 += loss_b.item()
        total_correct += get_num_correct(preds[0], y)

    print(
        "epoch", epoch, 
        #"total_correct:", total_correct, 
        "loss_a:", loss_a,
        "loss_b:", loss_b
    )

RuntimeError: Expected object of scalar type long int but got scalar type float for sequence element 1.